In [13]:
import sqlite3
from datetime import datetime

def registrar_presenca(matricula):
    # Conexão com o banco de dados SQLite
    conn = sqlite3.connect('dados_alunos.db')
    cursor = conn.cursor()
        
    # Obter a data atual no formato 'YYYY-MM-DD'
    data_atual = datetime.now().strftime('%d-%m-%y')
    coluna_entrada = f'{data_atual}-entrada'
    coluna_saida = f'{data_atual}-saida'
    horario_atual = datetime.now().strftime('%H:%M:%S')
        
    # Verificar se as colunas de entrada e saída existem, caso contrário, criar
    cursor.execute(f"PRAGMA table_info(alunos)")
    colunas = [info[1] for info in cursor.fetchall()]
        
    if coluna_entrada not in colunas:
        cursor.execute(f"ALTER TABLE alunos ADD COLUMN '{coluna_entrada}' TEXT")
        cursor.execute(f"UPDATE alunos SET '{coluna_entrada}' = 'F'")
        conn.commit()
        
    if coluna_saida not in colunas:
        cursor.execute(f"ALTER TABLE alunos ADD COLUMN '{coluna_saida}' TEXT")
        cursor.execute(f"UPDATE alunos SET '{coluna_saida}' = 'F'")
        conn.commit()
        
    # Registrar o horário de entrada ou saída
    query = f"""SELECT "{coluna_entrada}" FROM alunos WHERE Matricula = '{matricula}'"""
    cursor.execute(query)
    entrada = cursor.fetchone()
        
    if entrada[0] == 'F':
        cursor.execute(f"UPDATE alunos SET '{coluna_entrada}' = ? WHERE Matricula = ?", (horario_atual, matricula))
    else:
        cursor.execute(f"""SELECT "{coluna_saida}" FROM alunos WHERE Matricula = ?""", (matricula,))
        saida = cursor.fetchone()
            
        if saida[0] == 'F':
            cursor.execute(f"UPDATE alunos SET '{coluna_saida}' = ? WHERE Matricula = ?", (horario_atual, matricula))
        
    # Confirmar as mudanças e fechar a conexão
    conn.commit()
    conn.close()

registrar_presenca(12345678)
registrar_presenca(23456789)
registrar_presenca(34567890)
registrar_presenca(87654321)

('06:50:37',)
('06:50:44',)
('06:50:44',)
('06:50:44',)


In [14]:
#dar uma olhada se deu bom:

import pandas as pd
import sqlite3

conn = sqlite3.connect('dados_alunos.db')

df_total = pd.read_sql_query("SELECT * FROM alunos", conn)

print(df_total.head(5))

conn.close()

                       Nome Serie Turma  Matricula 18-06-24-entrada  \
0          Ana Paula Arósio    1ª     A   12345678         06:49:53   
1               Lulu Santos    3ª     C   23456789         06:50:37   
2              Renato Russo    2ª     A   34567890         06:50:37   
3  Angélica que vai de Táxi    2ª     B   87654321         06:50:37   
4       Monalisa da Pintura    1ª     D   98765432                F   

  18-06-24-saida  
0       06:50:37  
1       06:50:44  
2       06:50:44  
3       06:50:44  
4              F  
